# Спрос на рейсы авиакомпании

## Описание проекта

Вы аналитик российской авиакомпании F9, выполняющей внутренние пассажирские перевозки. Важно понять предпочтения пользователей, покупающих билеты на разные направления.
Вам предстоит изучить базу данных и проанализировать спрос пассажиров на рейсы в города, где проходят крупнейшие культурные фестивали. 

## Инструкция по выполнению проекта

1. Проведите исследовательский анализ данных средствами SQL.
2. Соберите данные для анализа из базы.
3. Проанализируйте данные средствами Python.

### Описание данных

В вашем распоряжении база данных об авиаперевозках.
Таблица airports — информация об аэропортах:

    airport_code — трёхбуквенный код аэропорта
    airport_name — название аэропорта
    city — город
    timezone — временная зона

Таблица aircrafts — информация о самолётах:

    aircraft_code — код модели самолёта
    model — модель самолёта
    range — дальность полёта

Таблица tickets — информация о билетах:

    ticket_no — уникальный номер билета
    passenger_id — персональный идентификатор пассажира
    passenger_name — имя и фамилия пассажира

Таблица flights — информация о рейсах:

    flight_id — уникальный идентификатор рейса
    departure_airport — аэропорт вылета
    departure_time — дата и время вылета
    arrival_airport — аэропорт прилёта
    arrival_time — дата и время прилёта
    aircraft_code — id самолёта

Таблица ticket_flights — стыковая таблица «рейсы-билеты»

    ticket_no — номер билета
    flight_id — идентификатор рейса

Таблица festivals — информация о фестивалях

    festival_id — уникальный номер фестиваля
    festival_date — дата проведения фестиваля
    festival_city — город проведения фестиваля
    festival_name — название фестиваля

### Схема таблиц

<img src='PK_FK_weather_1603199319.jpg'>

Пояснение: В базе данных нет прямой связи между таблицами *airports* и *festivals*, а также *festivals* и *flights*. Но вы можете писать JOIN и связывать эти таблицы по городу проведения фестиваля (*festival_city*) и городу аэропорта (*city*). Потребуется некоторое преобразование дат в *flights*, и тогда эту таблицу также можно будет связать по дате проведения фестиваля (*festival_date*) в запросах с JOIN. 

### Работа с базой

#### Найдите количество рейсов на каждой модели самолёта с вылетом в сентябре 2018 года. Назовите получившееся поле flights_amount и выведите его. Также напечатайте на экране поле model. Столбцы в таблице должны быть выведены в следующем порядке:

    model
    flights_amount

In [2]:
SELECT aircrafts.model, 
       COUNT(DISTINCT(flights.flight_id)) as flights_amount 
FROM aircrafts
     INNER JOIN flights on aircrafts.aircraft_code = flights.aircraft_code
WHERE CAST(flights.departure_time as date) >= '2018-09-01' AND CAST(flights.departure_time as date) < '2018-09-01'
GROUP BY aircrafts.model

SyntaxError: invalid syntax (<ipython-input-2-9d8a72bfe008>, line 1)

#### Посчитайте отправленные в сентябре рейсы самолётов следующих моделей:

    Boeing,
    Airbus,
    другие (“other”).

Назовите получившуюся переменную flights_amount и выведите её значение на экран.

In [ ]:
SELECT 
    CASE WHEN aircrafts.model LIKE '%Boeing%' THEN 'Boeing'
    WHEN aircrafts.model LIKE '%Airbus%' THEN 'Airbus'
    ELSE 'other'
    END AS model,
    COUNT(*) AS flights_amount
FROM 
    aircrafts
    INNER JOIN flights ON flights.aircraft_code = aircrafts.aircraft_code
WHERE
    CAST(flights.departure_time AS date) >= '2018-09-01' AND CAST(flights.departure_time AS date) < '2018-10-01'
GROUP BY
    CASE WHEN aircrafts.model LIKE '%Boeing%' THEN 'Boeing'
    WHEN aircrafts.model LIKE '%Airbus%' THEN 'Airbus'
    ELSE 'other'
    END

#### Посчитайте среднее количество прибывающих рейсов в день для каждого города за август 2018 года. Назовите получившееся поле average_flights, вместе с ним выведите столбец city.
Выводите столбцы в таком порядке:

    city,
    average_flights.

In [ ]:
SELECT 
    subq.city, 
    AVG(subq.average1_flights) as average_flights
FROM
(SELECT
    airports.city,
    COUNT(flights.flight_id) as average1_flights,
    EXTRACT(DAY FROM CAST(flights.arrival_time as date) as day
FROM flights
    INNER JOIN airports on flights.departure_airport = airports.airport_code
WHERE CAST(flights.arrival_time as date) BETWEEN '2018-01-01' AND '2018-01-31'
GROUP BY airports.city) as subq
GROUP BY airports.city

<img src='PK_FK_weather_1603199319.jpg'>

#### Установите фестивали, которые проходили с 23 июля по 30 сентября 2018 года в Москве, и номер недели, в которую они проходили. Выведите название фестиваля festival_name и номер недели festival_week.

In [ ]:
SELECT 
    festivals.festivals_name,
    EXTRACT(WEEK FROM festivals.festivals_date) as festival_week
FROM 
    festivals
WHERE
    festivals.festivals_date BETWEEN "2018-07-23" AND "2018-09-30"
    AND
    festivals.festivals_city = "Москва"

#### Для каждой недели с 23 июля по 30 сентября 2018 года посчитайте билеты, которые купили на рейсы в Москву (номер недели week_number и количество билетов ticket_amount). Получите таблицу:

    с количеством купленных за неделю билетов;
    отметкой, проходил ли в эту неделю фестиваль;
    название фестиваля festival_name;
    номер недели week_number.

Выводите столбцы в таком порядке: - week_number - ticket_amount - festival_week - festival_name

In [ ]:
SELECT
    EXTRACT(week FROM CAST(flights.arrival_time AS date)) AS week_number,
    COUNT(ticket_flights.ticket_no) AS ticket_amount,
    sub.festival_week AS festival_week,
    sub.festival_name AS festival_name
    
FROM tickets
    LEFT JOIN ticket_flights ON ticket_flights.ticket_no  = tickets.ticket_no
    LEFT JOIN flights ON flights.flight_id  = ticket_flights.flight_id
    LEFT JOIN airports ON airports.airport_code = flights.arrival_airport
    LEFT JOIN
        (SELECT
             festival_name,
             EXTRACT(WEEK FROM CAST(festival_date AS date)) AS festival_week
         FROM
             festivals
         WHERE
             festival_date BETWEEN '2018-07-23' AND '2018-09-30' AND festival_city = 'Москва')
             AS sub ON sub.festival_week = EXTRACT(week FROM CAST(flights.arrival_time AS date))
WHERE
    CAST(flights.arrival_time AS date) BETWEEN '2018-07-23' AND '2018-09-30' AND airports.city = 'Москва'
GROUP BY
    EXTRACT(week FROM CAST(flights.arrival_time AS date)), 
    festival_week, 
    festival_name